# constellation detection system
fully functioning training model

In [ ]:
AA

Updated YAML configuration at stars_constellations_dataset/constellation_data.yaml
Training YOLOv8s model...
New https://pypi.org/project/ultralytics/8.3.114 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.112  Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=stars_constellations_dataset/constellation_data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=constellation_detector12, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_strid

train: Scanning C:\Users\Dell\Downloads\Star Constellation\stars_constellations_dataset\labels\train.cache... 1530 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1530/1530 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 237.530.6 MB/s, size: 37.7 KB)


val: Scanning C:\Users\Dell\Downloads\Star Constellation\stars_constellations_dataset\labels\val.cache... 146 images, 0 backgrounds, 0 corrupt: 100%|██████████| 146/146 [00:00<?, ?it/s]


Plotting labels to runs\detect\constellation_detector12\labels.jpg... 
optimizer: Adam(lr=0.0005, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\constellation_detector12
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


10 epochs completed in 0.174 hours.
Optimizer stripped from runs\detect\constellation_detector12\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\constellation_detector12\weights\best.pt, 22.5MB

Validating runs\detect\constellation_detector12\weights\best.pt...
Ultralytics 8.3.112  Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
Model summary (fused): 72 layers, 11,131,776 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  2.83it/s]


                   all        146        561      0.875      0.931      0.943      0.551
                Aquila         28         28      0.945          1      0.994      0.642
                Bootes         34         34      0.925          1      0.995      0.644
           Canis Major         29         29      0.939          1      0.995      0.623
           Canis Minor         35         35      0.887      0.895      0.962      0.383
            Cassiopeia         49         49       0.86          1      0.976      0.561
                Cygnus         35         35       0.73      0.971      0.966      0.696
                Gemini         36         36      0.994      0.944      0.985      0.611
                   Leo         30         30      0.868          1      0.987      0.642
                  Lyra         41         41      0.759      0.976      0.975      0.462
                  Moon         40         40      0.741        0.8      0.772      0.319
                 Orio

val: Scanning C:\Users\Dell\Downloads\Star Constellation\stars_constellations_dataset\labels\val.cache... 146 images, 0 backgrounds, 0 corrupt: 100%|██████████| 146/146 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.86it/s]


                   all        146        561      0.878      0.929       0.94      0.571
                Aquila         28         28      0.966          1      0.994      0.668
                Bootes         34         34      0.925          1      0.995      0.667
           Canis Major         29         29      0.937          1      0.995      0.644
           Canis Minor         35         35      0.891      0.886       0.95       0.41
            Cassiopeia         49         49       0.86          1      0.976      0.586
                Cygnus         35         35       0.73      0.971       0.97      0.721
                Gemini         36         36          1      0.944      0.972      0.647
                   Leo         30         30      0.868          1      0.987      0.662
                  Lyra         41         41      0.758      0.976      0.975      0.485
                  Moon         40         40      0.759        0.8      0.791       0.35
                 Orio

testing cuda installation

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device name: {torch.cuda.get_device_name(0)}")

c:\Users\Dell\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


PyTorch version: 2.6.0+cu118
CUDA available: True
CUDA version: 11.8
GPU device name: NVIDIA GeForce GTX 1070


trying to get preprocessing functions integrated

In [19]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm
import shutil

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0005
MODEL_TYPE = "yolov8s.pt"  # Using the small variant (11.2M parameters)

# Define constellation classes (16 classes)
CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv8 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def create_yolov8_structure():
    # Create a new dataset directory for preprocessed data
    processed_dir = f"{DATASET_PATH}_processed"
    
    # Create proper YOLOv8 structure
    os.makedirs(f"{processed_dir}/images/train", exist_ok=True)
    os.makedirs(f"{processed_dir}/images/val", exist_ok=True)
    os.makedirs(f"{processed_dir}/images/test", exist_ok=True)
    os.makedirs(f"{processed_dir}/labels/train", exist_ok=True)
    os.makedirs(f"{processed_dir}/labels/val", exist_ok=True)
    os.makedirs(f"{processed_dir}/labels/test", exist_ok=True)
    
    return processed_dir

def preprocess_dataset(train_only=False):
    """
    Preprocess both images and labels, ensuring both are properly copied and aligned
    """
    # Define directory paths
    raw_images_dir = f"{DATASET_PATH}/images"
    raw_labels_dir = f"{DATASET_PATH}/labels"
    proc_images_dir = f"{DATASET_PATH}/preprocessed_images"
    proc_labels_dir = f"{DATASET_PATH}/preprocessed_labels"
    
    # Create directories
    os.makedirs(f"{proc_images_dir}/train", exist_ok=True)
    os.makedirs(f"{proc_labels_dir}/train", exist_ok=True)
    
    if not train_only:
        os.makedirs(f"{proc_images_dir}/val", exist_ok=True)
        os.makedirs(f"{proc_labels_dir}/val", exist_ok=True)
        os.makedirs(f"{proc_images_dir}/test", exist_ok=True)
        os.makedirs(f"{proc_labels_dir}/test", exist_ok=True)
    
    # Process training set
    print("Processing training images and labels...")
    process_subset(
        f"{raw_images_dir}/train", 
        f"{raw_labels_dir}/train",
        f"{proc_images_dir}/train", 
        f"{proc_labels_dir}/train", 
        augment=True
    )
    
    if not train_only:
        # Process validation set
        print("Processing validation images and labels...")
        process_subset(
            f"{raw_images_dir}/val", 
            f"{raw_labels_dir}/val",
            f"{proc_images_dir}/val", 
            f"{proc_labels_dir}/val", 
            augment=False
        )
        
        # Process test set
        print("Processing test images and labels...")
        process_subset(
            f"{raw_images_dir}/test", 
            f"{raw_labels_dir}/test",
            f"{proc_images_dir}/test", 
            f"{proc_labels_dir}/test", 
            augment=False
        )
    
    return proc_images_dir, proc_labels_dir

def process_subset(img_input_dir, label_input_dir, img_output_dir, label_output_dir, augment=False):
    """
    Process a subset of the dataset, handling both images and labels
    """
    # Get all image files
    image_files = [f for f in os.listdir(img_input_dir) if f.endswith(('.jpg', '.png', '.jpeg', '.tif', '.tiff'))]
    
    for img_file in tqdm(image_files, desc="Processing"):
        # Get image base name without extension
        img_base = os.path.splitext(img_file)[0]
        img_path = os.path.join(img_input_dir, img_file)
        label_file = f"{img_base}.txt"
        label_path = os.path.join(label_input_dir, label_file)
        
        # Skip if label doesn't exist
        if not os.path.exists(label_path):
            print(f"Warning: No label file found for {img_file}, skipping")
            continue
        
        # Read and preprocess image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Basic preprocessing
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        l, a, b = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2LAB))
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        l = clahe.apply(l)
        img = cv2.cvtColor(cv2.merge((l, a, b)), cv2.COLOR_LAB2RGB)
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        
        # Save preprocessed image
        img_output_path = os.path.join(img_output_dir, img_file)
        cv2.imwrite(img_output_path, cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR))
        
        # Copy label file (no transformation needed for basic resize)
        label_output_path = os.path.join(label_output_dir, label_file)
        shutil.copy(label_path, label_output_path)
        
        # Simple augmentation
        if augment:
            # Rotation 90 degrees
            height, width = img_resized.shape[:2]
            matrix = cv2.getRotationMatrix2D((width/2, height/2), 90, 1)
            rotated = cv2.warpAffine(img_resized, matrix, (width, height))
            rot_img_path = os.path.join(img_output_dir, f"rot_90_{img_file}")
            cv2.imwrite(rot_img_path, cv2.cvtColor(rotated, cv2.COLOR_RGB2BGR))
            
            # Transform labels for rotation (this is the tricky part)
            rot_label_path = os.path.join(label_output_dir, f"rot_90_{img_base}.txt")
            with open(label_path, 'r') as f_in, open(rot_label_path, 'w') as f_out:
                for line in f_in:
                    parts = line.strip().split()
                    if len(parts) >= 5:  # class x y width height
                        cls = parts[0]
                        x, y = float(parts[1]), float(parts[2])
                        w, h = float(parts[3]), float(parts[4])
                        
                        # For 90 degree rotation: (x,y) -> (1-y,x)
                        new_x = 1.0 - y
                        new_y = x
                        # Swap width and height
                        new_w, new_h = h, w
                        
                        f_out.write(f"{cls} {new_x:.6f} {new_y:.6f} {new_w:.6f} {new_h:.6f}\n")
    
    return
    

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config(dataset_path):
    config = {
        'path': os.path.abspath(dataset_path),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{dataset_path}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    return f"{dataset_path}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv8s model using the dataset
    """
    # Load a pretrained YOLOv8s model
    model = YOLO(MODEL_TYPE)
    
    # Train the model
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="Adam",  # As per requirements
        lr0=LEARNING_RATE,  # Initial learning rate
        conf=0.25,  # Detection confidence threshold
        name="constellation_detector",
        device=0 if torch.cuda.is_available() else 'cpu'
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    # Validate the model
    results = model.val(conf=0.1)
    
    # Handle potentially multi-element arrays by taking the first element if needed
    def safe_convert(value):
        try:
            # Try direct conversion first
            return float(value)
        except (TypeError, ValueError):
            # If that fails, try to access the first element
            if hasattr(value, '__len__') and len(value) > 0:
                return float(value[0])
            # If that also fails, return a default value
            return 0.0
    
    metrics = {
        "mAP50": safe_convert(results.box.map50),
        "precision": safe_convert(results.box.p),
        "recall": safe_convert(results.box.r),
        "f1": safe_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"mAP50: {metrics['mAP50']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image
    """
    # Run inference
    results = model.predict(image_path, conf=0.25)
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    # Create new dataset structure with proper YOLOv8 format
    processed_dir = create_yolov8_structure()
    
    # Preprocess and copy data from original dataset to new structure
    print("Preprocessing dataset...")
    
    # Process training set - copy from original to new structure
    process_subset(
        f"{DATASET_PATH}/images/train", 
        f"{DATASET_PATH}/labels/train",
        f"{processed_dir}/images/train", 
        f"{processed_dir}/labels/train", 
        augment=True
    )
    
    # Process validation set
    process_subset(
        f"{DATASET_PATH}/images/val", 
        f"{DATASET_PATH}/labels/val",
        f"{processed_dir}/images/val", 
        f"{processed_dir}/labels/val", 
        augment=False
    )
    
    # Process test set
    process_subset(
        f"{DATASET_PATH}/images/test", 
        f"{DATASET_PATH}/labels/test",
        f"{processed_dir}/images/test", 
        f"{processed_dir}/labels/test", 
        augment=False
    )
    
    # Create YAML config for the new structure
    yaml_path = create_yaml_config(processed_dir)
    
    # Train using the properly structured preprocessed dataset
    print("Training YOLOv8s model...")
    model = YOLO(MODEL_TYPE)
    
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="Adam",  
        lr0=LEARNING_RATE,
        name="constellation_detector_processed"
    )
    
    # Evaluate and save the model
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    model_save_path = "models/constellation_detector_processed_yolov8s.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")
    
    print("Processed constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

Preprocessing dataset...


Processing: 100%|██████████| 74/74 [00:03<00:00, 21.62it/s]


Training YOLOv8s model...
New https://pypi.org/project/ultralytics/8.3.114 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.112  Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=stars_constellations_dataset_processed/constellation_data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=constellation_detector_processed, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnosti

train: Scanning C:\Users\Dell\Downloads\Star Constellation\stars_constellations_dataset_processed\labels\train... 3060 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3060/3060 [00:05<00:00, 556.63it/s]


train: New cache created: C:\Users\Dell\Downloads\Star Constellation\stars_constellations_dataset_processed\labels\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 6.81.2 MB/s, size: 46.5 KB)


...


10 epochs completed in 0.298 hours.
Optimizer stripped from runs\detect\constellation_detector_processed\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\constellation_detector_processed\weights\best.pt, 22.5MB

Validating runs\detect\constellation_detector_processed\weights\best.pt...
Ultralytics 8.3.112  Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
Model summary (fused): 72 layers, 11,131,776 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.96it/s]


                   all        146        561      0.885      0.933       0.95      0.559
                Aquila         28         28      0.906      0.964      0.941      0.597
                Bootes         34         34      0.874          1      0.995      0.626
           Canis Major         29         29      0.957          1      0.995       0.68
           Canis Minor         35         35      0.948      0.914      0.966      0.397
            Cassiopeia         49         49      0.884          1      0.938      0.545
                Cygnus         35         35      0.758      0.971      0.941      0.682
                Gemini         36         36      0.972      0.956      0.994      0.597
                   Leo         30         30      0.866          1      0.991      0.646
                  Lyra         41         41      0.711          1      0.973       0.52
                  Moon         40         40       0.85        0.9       0.92      0.359
                 Orio

val: Scanning C:\Users\Dell\Downloads\Star Constellation\stars_constellations_dataset_processed\labels\val.cache... 146 images, 0 backgrounds, 0 corrupt: 100%|██████████| 146/146 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.07it/s]


                   all        146        561      0.885      0.931      0.949      0.583
                Aquila         28         28      0.907      0.964      0.951      0.621
                Bootes         34         34      0.874          1      0.995      0.643
           Canis Major         29         29      0.958          1      0.995      0.705
           Canis Minor         35         35      0.949      0.914       0.96      0.425
            Cassiopeia         49         49      0.884          1      0.938      0.567
                Cygnus         35         35      0.758      0.971      0.935      0.699
                Gemini         36         36      0.972      0.955      0.994       0.63
                   Leo         30         30      0.866          1       0.99      0.676
                  Lyra         41         41      0.725          1      0.974      0.539
                  Moon         40         40      0.856      0.894       0.93       0.38
                 Orio